In this notebook, I will walk you through the steps to be taken to clean and carry out exploratory data analysis on the IMDB TV series data from kaggle https://www.kaggle.com/datasets/suraj520/imdb-tv-series-data
The dataset contains information about TV series from IMDb, including details such as title, IMDb ID, release year, genre, cast, synopsis, rating, runtime, certificate, number of votes, and gross revenue. The data is scraped from the IMDb website using web scraping techniques and is organized into separate CSV files for each genre.

Importing Required Libraries:
* glob: This library is used to retrieve file paths matching specified patterns.
* pandas: It provides data manipulation and analysis capabilities.
* os: This library provides a way to interact with the operating system, including file and directory operations.
* zipfile: This library offers tools to create, read, write, and extract files from ZIP archives.

In [ ]:
import glob
import pandas as pd
import os
from zipfile import ZipFile

The imdb data is a zip file called archive.zip, to find it, you start with an empty list called zip_files to store the paths of the identified ZIP files.
Begin walking through the directory structure using os.walk("/home/anees/projects/EDA_and_data_cleaning").
For each directory, examine the files within it.
If a file has a ".zip" extension, add its path to the zip_files list.
Continue the process until all directories have been traversed.
Finally, print the zip_files list to display the paths of all identified ZIP files.

In [ ]:
zip_files = []
for root, dirs, files in os.walk("/home/anees/projects/EDA_and_data_cleaning"):
    for file in files:
        if file.endswith(".zip"):
            zip_files.append(os.path.join(root, file))

print(zip_files)

After obtaining the list of ZIP file paths, you will notice that the "archive.zip" file is located at the last index. Utilizing the zipfile library, you can extract the contents of this ZIP file to a folder named "imdb_files" within the "raw_data/imdb" directory. Once extracted, you can verify the extraction process by printing the paths of the extracted CSV files.

In [ ]:
with ZipFile(zip_files[-1], "r") as file:
    file.extractall(path="/home/anees/projects/EDA_and_data_cleaning/raw_data/imdb/imdb_files")

csv_files = glob.glob("/home/anees/projects/EDA_and_data_cleaning/raw_data/imdb/imdb_files/*.csv")
for csv_file in csv_files:
    print(csv_file)

To process the extracted CSV files and create a consolidated DataFrame, you can iterate through the list of CSV file paths, read each CSV file using pd.read_csv(), and append the resulting DataFrames to a list called dataframes. Finally, you can use pd.concat() to concatenate the DataFrames into a single DataFrame called imdb_df.
By executing this code, you will obtain the consolidated DataFrame imdb_df, which contains the data from all the extracted CSV files.

In [ ]:
dataframes = []
for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    dataframes.append(df)
imdb_df = pd.concat(dataframes, ignore_index=True)

To retrieve information about the structure and summary of the consolidated DataFrame imdb_df, you can use the info() method.
Here's the code:

In [ ]:
imdb_df.info() 

The imdb_df.info() command provides you with information about the structure and summary of the consolidated DataFrame imdb_df. 
The output provides the following details:
* The DataFrame has a RangeIndex with 236,828 entries, ranging from index 0 to index 236,827.
* There are 11 columns in the DataFrame.
* Each column is listed along with its non-null count and data type.
* The DataFrame contains a mix of data types, with 10 columns being of type object and 1 column being of type float64.
* The memory usage of the DataFrame is reported as approximately 19.9+ MB.
This information helps you understand the composition and structure of the DataFrame, including the number of entries, data types, and missing values for each column.

To enhance the DataFrame's structure, you can modify the column names to adhere to professional conventions.

You can make the column names consistent by replacing any spaces with underscores (_) and converting them to lowercase. You can then use the info() method again to verify the changes have been effected

Consider the following code:

In [ ]:
imdb_df.columns = imdb_df.columns.str.replace(' ', '_').str.lower()
imdb_df.info()

To preview the data in the DataFrame, you can use the head() and tail(), and sample() methods. By defualt the head() method shows the top 5 rows of the DataFrame, the tail() method shows the bottom 5 rows and sample() method returns a single random row from the DataFrame. These methods can take an argument specifying the number of rows to be returned.
For example, imdb_df.head(10) will display the top 10 rows, imdb_df.tail(3) will display the last 3 rows and imdb_df.sample(6) randomly selects and returns 6 rows from the DataFrame.

By utilizing these methods, you can preview the data in the DataFrame and get a sense of its contents and structure.

In [ ]:
imdb_df.head()

In [ ]:
imdb_df.tail()

In [ ]:
imdb_df.sample(5)

One thing you have to decide very early on in a data cleaning and EDA process is identify your index column, an index column provides labels or names for the rows in the DataFrame. It allows you to uniquely identify and access specific rows based on their index values. If you don't specify an index column, pandas assigns a default integer index starting from 0. However, setting a meaningful and appropriate index can enhance data analysis and manipulation capabilities.

Looking at the output of the info, head, tail and sample methods, one colunm that looks like it can be use as an index column is imdb_id column, lets explore further. to see the content of the column, use the the code below

In [ ]:
imdb_df["imdb_id"].sample(10)

also since you want the column to uniqely identify each row in the dataframe, you have to check and make sure there is no duplicate value in the column

In [ ]:
imdb_df["imdb_id"].duplicated().sum()

there are 127631 duplicate values in the column, you con confirm if they are actual duplicates or if the same imdb_id is associated to different records

In [ ]:
imdb_df.head(100).duplicated()

using the head method to select the first 100 rows shows no duplicates, lets use the tail method instead

In [ ]:
imdb_df.tail(100).duplicated()

the tail method in the other hand shows a couple of records are duplicated, selct the last record on the dataframe using the iloc method

In [ ]:
last_row = imdb_df.iloc[-1]
print(last_row)

In [ ]:
imdb_df[imdb_df["title"] == "Evil Dead"]

from the result, we can see that they are actual duplicate, to be safe, lets check for another row

In [ ]:
second_last_row = imdb_df.iloc[-2]
print(second_last_row)

In [ ]:
imdb_df[imdb_df["title"] == 'Scream']

even though there are multiple movies with the title scream, they all have different imdb_id, so we can conclude that each imdb_id is associated with only one unique record. Now we can safely drop any duplicate record in the column

In [ ]:
imdb_df = imdb_df.drop_duplicates(subset="imdb_id")


you then verify if the duplicates have been dropped.

In [ ]:
imdb_df["imdb_id"].duplicated().sum()

this time around, you got zero, meaning there are no duplicate value in the column. now we can safely use the imdb_id column as our index column

In [ ]:
imdb_df.set_index("imdb_id", inplace=True)

In [ ]:
imdb_df.head()

In [ ]:
print(imdb_df["title"].dtypes)

Something else you might have noticed is that the release_year column contain charcaters that are notthe release_year column should contain only four digit, signifying a year but as you can see, we have cases of special charcaters (braces and dash), and a case of the roman numeral "I" making an appearence 
To gain a comprehensive understanding of the cleanliness of the "release_year" column, you can identify several issues by examining its unique values.
To obtain a clear picture of the unique values in the column and address these concerns, we can assign the unique values to a variable and print them. Here's the code to accomplish this:

In [ ]:
unique_values = imdb_df['release_year'].unique()
#print(unique_values)
#optionally, you can loop through the list object and print out each unique value
for value in unique_values:
    print(value)

When addressing the task of cleaning the release_year column, you have two viable approaches. The first option entails directly cleaning the column itself, while the second option involves cleaning the unique_values and subsequently replacing the release_year column with the cleaned list.

The choice between these options largely depends on the specific requirements and constraints of the data analysis task at hand. Cleaning the column directly allows for immediate modifications within the DataFrame, which can be advantageous when there is a need to retain the original structure and integrity of the dataset. Conversely, opting to clean the unique_values list independently can offer the advantage of decoupling the cleaning process from the DataFrame, facilitating analysis and transformations on a reduced and sanitized dataset.

We will be going with the first approach, you can opt for the second one if you like.

You can clean the 'release_year' column by removing all characters that are not digits. The following code accomplishes this
The code uses the str.replace() method with the regular expression pattern r'[^\d]' to match any character that is not a digit. The regex=True parameter ensures that the replacement is performed using regular expressions. By replacing the matched characters with an empty string, you effectively remove all non-digit characters from the 'release_year' column.

After executing the code, the 'release_year' column will only contain the cleaned numerical values.

In [ ]:
imdb_df['release_year'] = imdb_df['release_year'].str.replace(r'[^\d]', '', regex=True)
imdb_df.head()

The resulting 'release_year' column is expected to display cleaned numerical values upon executing the code.
However, it is important to note that the 'release_year' column may still exhibit an issue as identified when inspecting the tail of the DataFrame. The removal of non-digit characters inadvertently eliminated the separator (-) that differentiates the two years in some entries.

In [ ]:
imdb_df.tail()

To address this problem, you can use regular expressions to insert the hyphen at the desired position. Here's an example:
In the following code, we use the str.replace() method with a regular expression pattern (\d{4})(\d{4}) to match the string of eight digits representing two years. The pattern captures the first four digits and the next four digits separately. Then, we replace the match with \1-\2, which inserts a hyphen '-' between the two captured groups.

In [ ]:
imdb_df['release_year'] = imdb_df['release_year'].str.replace(r'(\d{4})(\d{4})', r'\1-\2', regex=True)
imdb_df.tail()

You can proceed to drop any columns that are not relevant to your analysis. For example, if you are not interested in the imdb_id or the synopsis of each movie, you can drop those columns using code block below.
The code block drops the imdb_id and synopsis columns from the imdb_df DataFrame. The axis=1 argument tells Pandas to drop the columns along the column axis. The inplace=True argument tells Pandas to modify the original DataFrame.

In [ ]:
imdb_df.drop(["imdb_id", "synopsis"], axis = 1, inplace = True)
imdb_df.head(2)

Moving on, you can check for the total number of duplicate values in the dataframe with the following

In [ ]:
imdb_df.duplicated().sum()

In [ ]:
imdb_df["title"].value_counts()

In [ ]:
imdb_df[imdb_df['title'] == "Boys"]

In [ ]:
imdb_df.tail()